In [1]:
import geojson
import netCDF4 as nc
import os
import numpy as np

In [2]:
folder = r'p:\11205479-coclico\data\03_sea_level_jrc'
#dataset = 'CoastAlRisk_Europe_ESL_RCP45.nc'
#dataset = 'CoastAlRisk_Europe_ESL_RCP85.nc'
dataset = 'CoastAlRisk_Europe_ESL_Historical.nc'

In [3]:
ds = nc.Dataset(os.path.join(folder, dataset))
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    title: Extreme Sea Levels
    Institution: Joint European Research Center, Directorate for Space, Security and Migration, Via Enrico Fermi 2749, I-21027-Ispra
    Project Name: Prototype of a first Global Integrated Coastal Impact-based Flood Alert and Risk Assessment Tool
    Project Acronym: CoastAlRisk
    reference:  Vousdoukas MI., Mentaschi L., Voukouvalas E., Verlaan M., Feyen L. (2017). Extreme sea levels on the rise along Europe's coasts. Earth's Future. doi:10.1002/2016EF000505 
    email: michail.vousdoukas@ec.europa.eu
    version: 1.2
    terms_for_use: European Union, 1995-2015.
Reuse is authorised, provided the source is acknowledged. The reuse policy of the European Commission is implemented by a Decision of 12 December 2011.
    disclaimer: Unless the following would not be permitted or valid under applicable law, the following applies to the data/information provi

In [4]:
# get minima and maxima for the colormap boundaries
rp = ds['rp'][:].filled()

for idx, i in enumerate(rp):
    print(i, min(ds['esl'][:,idx,1].filled().flatten()), max(ds['esl'][:,idx,1].filled().flatten()))

5.0 0.4650539656031104 11.498925673926207
10.0 0.48766543080387137 11.634988892726845
20.0 0.5080505881690962 11.7618644273287
50.0 0.5322279771503171 11.91668998439283
100.0 0.548861282091625 12.024880099388861
200.0 0.5644194160947287 12.12601905899922
500.0 0.5838590944770807 12.24978115303037
1000.0 0.5981188276383687 12.336494467645258


In [5]:
# write data to file 
rp = ds['rp'][:].filled()

features = []
for j, (lon, lat) in enumerate(zip(ds['longitude'][:], ds['latitude'][:])):
    point = geojson.Point((float(lon), float(lat)))
    feature = geojson.Feature(geometry=point)
    feature['properties']['locationId'] = j
    feature['properties']['esl'] = np.array2string(ds['esl'][j][:,1].filled(), max_line_width=np.inf)
    feature['properties']['rp'] = str(rp)
    features.append(feature) 

# store the features
collection = geojson.FeatureCollection(features)
with open(os.path.join(folder, 'platform', r'EU_ESL_JRC_Hist_RP.geojson'), 'w') as f:
    geojson.dump(collection, f)

In [6]:
# check geojson content
with open(os.path.join(folder, 'platform', 'EU_ESL_JRC_Hist_RP.geojson')) as f:
    check = geojson.load(f)
    
check.keys()
check['features'][0]

{"geometry": {"coordinates": [-8.991017, 37.030349], "type": "Point"}, "properties": {"esl": "[3.30205426 3.41486462 3.52046817 3.64998335 3.74099187 3.82651434 3.93185498 4.00618858]", "locationId": 0, "rp": "[   5.   10.   20.   50.  100.  200.  500. 1000.]"}, "type": "Feature"}